## 1. Re-Calculate analysis period

##### Motivation:
- Among CI projects we found 76 without bugs in the selected period, however  with bugs in other periods.
- Several projects adopt CI service, but start doing automated builds and monitoring coverage only some time after. Therefore we can supose that indeed such projects employ CI on that date.

##### Procedure:
- We now consider the date of first coverage record for projects using Travis CI. This is due our criteria in consider CI projects those having automated builds and coverage metric.
- Columns were added to Projects table:
  - cov_init
  - cov_init_days
  - ci_service_init
  - ci_service_init_days
- Redifine analysis_point_days for projects using CI Service
- Get the new median days for CI projects (having cov_init not null) and set it on NO CI projects
- Redifine analysis_init and analysis_finish for all projects

##### Reminds:
- Re-execute merge_conflict process

##### 1.1 Setting Cov_init and Cov_init_days

- Get first coverage record to set cov_init
- Diff between days (repo_creation and cov_init) to set cov_init_days

In [92]:
repos = getProjectsToRecalculate()

i=1
j=0
for repo in repos: 
    project = repo[0]
    repo_creation = repo[1]
    print('Processing Dates: {}\t\t {}\{}  -  {}'.format(project,i,len(repos),datetime.now().strftime("%d/%m/%y %H:%M:%S")))
    i+=1
    
    cov_init = getFirstCov(project)[0]
    build_init = getFirstBuild(project)[0]
    
    if cov_init is not None and build_init is not None and cov_init < build_init:
        cov_init = build_init
    
    print(cov_init)
    
    if cov_init is not None:
        days = abs(cov_init.replace(tzinfo=None) - repo_creation.replace(tzinfo=None)).days
        setCovInit(project,cov_init,days)
    

Processing Dates: actorapp/actor-platform		 1\420  -  21/06/23 15:41:28
None
Processing Dates: adambard/learnxinyminutes-docs		 3\420  -  21/06/23 15:41:28
None
Processing Dates: AdAway/AdAway		 5\420  -  21/06/23 15:41:29
None
Processing Dates: adobe/brackets		 7\420  -  21/06/23 15:41:29
None
Processing Dates: agentejo/cockpit		 9\420  -  21/06/23 15:41:29
None
Processing Dates: airbnb/epoxy		 11\420  -  21/06/23 15:41:29
None
Processing Dates: airbnb/lottie-android		 13\420  -  21/06/23 15:41:29
None
Processing Dates: airbnb/visx		 15\420  -  21/06/23 15:41:29
2020-09-21 20:02:55
Processing Dates: ajaxorg/ace		 17\420  -  21/06/23 15:41:30
Contagem de mudanças... 1
2019-08-03 11:03:25
Processing Dates: akeneo/pim-community-dev		 19\420  -  21/06/23 15:41:30
None
Processing Dates: akka/alpakka		 21\420  -  21/06/23 15:41:30
None
Processing Dates: Alamofire/Alamofire		 23\420  -  21/06/23 15:41:30
None
Processing Dates: albumentations-team/albumentations		 25\420  -  21/06/23 15:41:30

Contagem de mudanças... 13
2021-05-11 17:41:20
Processing Dates: cloudfoundry/cli		 209\420  -  21/06/23 15:41:46
None
Processing Dates: coala/coala		 211\420  -  21/06/23 15:41:46
None
Processing Dates: code4craft/webmagic		 213\420  -  21/06/23 15:41:47
None
Processing Dates: coleifer/peewee		 215\420  -  21/06/23 15:41:47
None
Processing Dates: collectd/collectd		 217\420  -  21/06/23 15:41:47
None
Processing Dates: comfy/comfortable-mexican-sofa		 219\420  -  21/06/23 15:41:47
Contagem de mudanças... 14
2017-12-23 22:46:50
Processing Dates: Conflux-Chain/conflux-rust		 221\420  -  21/06/23 15:41:47
None
Processing Dates: criteo/cuttle		 223\420  -  21/06/23 15:41:48
None
Processing Dates: croogo/croogo		 225\420  -  21/06/23 15:41:48
Contagem de mudanças... 15
2017-09-12 09:25:08
Processing Dates: CyC2018/CS-Notes		 227\420  -  21/06/23 15:41:48
None
Processing Dates: CymChad/BaseRecyclerViewAdapterHelper		 229\420  -  21/06/23 15:41:48
None
Processing Dates: Daltron/NotificationBa

None
Processing Dates: jshint/jshint		 421\420  -  21/06/23 15:42:02
Contagem de mudanças... 25
2019-08-20 16:44:03
Processing Dates: k3s-io/k3s		 423\420  -  21/06/23 15:42:02
None
Processing Dates: Kapeli/Dash-iOS		 425\420  -  21/06/23 15:42:03
None
Processing Dates: kbengine/kbengine		 427\420  -  21/06/23 15:42:03
None
Processing Dates: kirki-framework/kirki		 429\420  -  21/06/23 15:42:03
None
Processing Dates: knative/serving		 431\420  -  21/06/23 15:42:03
None
Processing Dates: koding/koding		 433\420  -  21/06/23 15:42:03
None
Processing Dates: koenbok/Framer		 435\420  -  21/06/23 15:42:04
None
Processing Dates: kubernetes/client-go		 437\420  -  21/06/23 15:42:04
None
Processing Dates: kubernetes-client/python		 439\420  -  21/06/23 15:42:04
None
Processing Dates: kubernetes/ingress-nginx		 441\420  -  21/06/23 15:42:04
Contagem de mudanças... 26
2019-02-14 13:29:57
Processing Dates: kubernetes/kubernetes		 443\420  -  21/06/23 15:42:04
2015-12-23 01:31:32
Processing Dates:

None
Processing Dates: phpbrew/phpbrew		 623\420  -  21/06/23 15:42:18
Contagem de mudanças... 41
2018-02-15 20:25:21
Processing Dates: phpseclib/phpseclib		 625\420  -  21/06/23 15:42:18
None
Processing Dates: phpservermon/phpservermon		 627\420  -  21/06/23 15:42:18
None
Processing Dates: PierfrancescoSoffritti/android-youtube-player		 629\420  -  21/06/23 15:42:18
None
Processing Dates: pilosa/pilosa		 631\420  -  21/06/23 15:42:18
Contagem de mudanças... 42
2018-07-09 17:54:04
Processing Dates: pomerium/pomerium		 633\420  -  21/06/23 15:42:18
2021-06-08 18:11:31
Processing Dates: presidentbeef/brakeman		 635\420  -  21/06/23 15:42:19
Contagem de mudanças... 43
2019-02-04 19:43:19
Processing Dates: progit/progit		 637\420  -  21/06/23 15:42:19
None
Processing Dates: prometheus/node_exporter		 639\420  -  21/06/23 15:42:19
None
Processing Dates: Proxmark/proxmark3		 641\420  -  21/06/23 15:42:19
None
Processing Dates: psi-probe/psi-probe		 643\420  -  21/06/23 15:42:19
Contagem de m

None
Processing Dates: x-extends/vxe-table		 823\420  -  21/06/23 15:42:34
None
Processing Dates: xmartlabs/Eureka		 825\420  -  21/06/23 15:42:34
None
Processing Dates: yabwe/medium-editor		 827\420  -  21/06/23 15:42:34
Contagem de mudanças... 57
2016-07-13 19:23:24
Processing Dates: yarnpkg/yarn		 829\420  -  21/06/23 15:42:34
None
Processing Dates: yzhao062/pyod		 831\420  -  21/06/23 15:42:34
Contagem de mudanças... 58
2019-12-22 22:25:13
Processing Dates: zendesk/maxwell		 833\420  -  21/06/23 15:42:34
None
Processing Dates: zixun/GodEye		 835\420  -  21/06/23 15:42:34
None
Processing Dates: z-song/laravel-admin		 837\420  -  21/06/23 15:42:35
None
Processing Dates: zyedidia/micro		 839\420  -  21/06/23 15:42:35
None


##### 1.2 Redifining analysis_point and analysis_point_days for projects using CI Service

- We ran the SQL:
   - update projects set analysis_point_days = cov_init_days, analysis_point = cov_init where ci_service like 'Travis CI';

##### 1.3. Get the new median days for CI projects (having cov_init not null) and set it on NO CI projects

- Median of analysis point (Age on CI adoption)
   - In terms of coverage and build practices
- Process the median for CI projects (those having builds and coverages)
- Set NON-CI projects with the median, all of them.

In [99]:
medianAge = getMedianCIAdoption()[0][0]
print('Age median for CI adoption: {} days.'.format(medianAge))

print('Setting this median as analysis_point for all NON-CI projects.')
setNonCIProjectAnalysis_Point_Days(medianAge)
print('Done.')

repos = getNonCIProjects()
i=0
for repo in repos:  
    project = repo[0]
    created = repo[1]

    if created is not None:
        analysis_point = created + relativedelta(days=medianAge)

        i += 1
        print('{}/{} - {} Project {} - Created: {} - Days: {} - Analysis: {}'.format(i,len(repos),datetime.now().strftime("%H:%M:%S"),project,created,medianAge,analysis_point))

        setProjectAnalysis_Point(project,analysis_point)


Age median for CI adoption: 212.0 days.
Setting this median as analysis_point for all NON-CI projects.
Done.
1/743 - 15:51:46 Project 201206030/fiction_house - Created: 2019-11-10 22:13:52-03:00 - Days: 212.0 - Analysis: 2020-06-09 22:13:52-03:00
2/743 - 15:51:46 Project 5iux/sou - Created: 2019-01-23 03:37:28-03:00 - Days: 212.0 - Analysis: 2019-08-23 03:37:28-03:00
3/743 - 15:51:46 Project 81813780/AVLoadingIndicatorView - Created: 2015-10-24 17:18:33-03:00 - Days: 212.0 - Analysis: 2016-05-23 17:18:33-03:00
4/743 - 15:51:46 Project aa112901/remusic - Created: 2016-03-24 03:20:39-03:00 - Days: 212.0 - Analysis: 2016-10-22 03:20:39-03:00
5/743 - 15:51:46 Project AAChartModel/AAChartKit - Created: 2017-03-16 22:39:04-03:00 - Days: 212.0 - Analysis: 2017-10-14 22:39:04-03:00
6/743 - 15:51:46 Project aaPanel/BaoTa - Created: 2019-06-17 07:59:46-03:00 - Days: 212.0 - Analysis: 2020-01-15 07:59:46-03:00
7/743 - 15:51:46 Project AceLewis/my_first_calculator.py - Created: 2016-05-06 14:48:51

72/743 - 15:51:47 Project calebwin/emu - Created: 2019-03-28 09:07:40-03:00 - Days: 212.0 - Analysis: 2019-10-26 09:07:40-03:00
73/743 - 15:51:47 Project cas-bigdatalab/piflow - Created: 2018-05-02 22:47:46-03:00 - Days: 212.0 - Analysis: 2018-11-30 22:47:46-03:00
74/743 - 15:51:47 Project Ccixyj/JBusDriver - Created: 2017-04-09 11:35:49-03:00 - Days: 212.0 - Analysis: 2017-11-07 11:35:49-03:00
75/743 - 15:51:47 Project celzero/rethink-app - Created: 2020-06-08 10:41:18-03:00 - Days: 212.0 - Analysis: 2021-01-06 10:41:18-03:00
76/743 - 15:51:47 Project CharlesPikachu/Games - Created: 2018-06-13 05:31:26-03:00 - Days: 212.0 - Analysis: 2019-01-11 05:31:26-03:00
77/743 - 15:51:47 Project CharlinFeng/CorePhotoBroswerVC - Created: 2015-05-10 01:36:43-03:00 - Days: 212.0 - Analysis: 2015-12-08 01:36:43-03:00
78/743 - 15:51:47 Project chat-sdk/chat-sdk-ios - Created: 2016-12-20 11:02:20-03:00 - Days: 212.0 - Analysis: 2017-07-20 11:02:20-03:00
79/743 - 15:51:47 Project ChavezChen/CWLateralSl

143/743 - 15:51:48 Project Dman95/SASM - Created: 2013-06-03 17:46:24-03:00 - Days: 212.0 - Analysis: 2014-01-01 17:46:24-03:00
144/743 - 15:51:48 Project docker-archive/deploykit - Created: 2016-04-08 18:43:02-03:00 - Days: 212.0 - Analysis: 2016-11-06 18:43:02-03:00
145/743 - 15:51:48 Project Dod-o/Statistical-Learning-Method_Code - Created: 2018-11-15 12:33:17-03:00 - Days: 212.0 - Analysis: 2019-06-15 12:33:17-03:00
146/743 - 15:51:48 Project dog-qiuqiu/Yolo-Fastest - Created: 2020-08-21 00:37:01-03:00 - Days: 212.0 - Analysis: 2021-03-21 00:37:01-03:00
147/743 - 15:51:48 Project Doikki/DKVideoPlayer - Created: 2017-04-10 22:33:38-03:00 - Days: 212.0 - Analysis: 2017-11-08 22:33:38-03:00
148/743 - 15:51:48 Project drehimself/laravel-ecommerce-example - Created: 2017-12-11 07:04:02-03:00 - Days: 212.0 - Analysis: 2018-07-11 07:04:02-03:00
149/743 - 15:51:48 Project duoergun0729/1book - Created: 2017-05-15 19:30:40-03:00 - Days: 212.0 - Analysis: 2017-12-13 19:30:40-03:00
150/743 - 1

206/743 - 15:51:49 Project goabstract/Awesome-Design-Tools - Created: 2019-02-12 10:55:30-03:00 - Days: 212.0 - Analysis: 2019-09-12 10:55:30-03:00
207/743 - 15:51:49 Project GoesToEleven/GolangTraining - Created: 2015-08-09 20:09:58-03:00 - Days: 212.0 - Analysis: 2016-03-08 20:09:58-03:00
208/743 - 15:51:49 Project GoesToEleven/golang-web-dev - Created: 2016-08-24 19:50:00-03:00 - Days: 212.0 - Analysis: 2017-03-24 19:50:00-03:00
209/743 - 15:51:49 Project googlearchive/android-ConstraintLayoutExamples - Created: 2017-03-01 21:11:54-03:00 - Days: 212.0 - Analysis: 2017-09-29 21:11:54-03:00
210/743 - 15:51:49 Project googlearchive/code-prettify - Created: 2015-04-22 15:32:17-03:00 - Days: 212.0 - Analysis: 2015-11-20 15:32:17-03:00
211/743 - 15:51:49 Project googlearchive/ios-swift-chat-example - Created: 2014-08-13 18:55:07-03:00 - Days: 212.0 - Analysis: 2015-03-13 18:55:07-03:00
212/743 - 15:51:49 Project googleforgames/open-match - Created: 2018-08-16 20:57:53-03:00 - Days: 212.0 

278/743 - 15:51:49 Project Jasonchenlijian/FastBle - Created: 2016-08-22 23:06:33-03:00 - Days: 212.0 - Analysis: 2017-03-22 23:06:33-03:00
279/743 - 15:51:49 Project jathu/UIImageColors - Created: 2015-06-19 05:00:52-03:00 - Days: 212.0 - Analysis: 2016-01-17 05:00:52-03:00
280/743 - 15:51:49 Project java-aodeng/hope-boot - Created: 2018-10-17 04:50:48-03:00 - Days: 212.0 - Analysis: 2019-05-17 04:50:48-03:00
281/743 - 15:51:49 Project JavaNoober/BackgroundLibrary - Created: 2018-09-09 05:52:45-03:00 - Days: 212.0 - Analysis: 2019-04-09 05:52:45-03:00
282/743 - 15:51:49 Project jbangert/trapcc - Created: 2012-12-30 09:28:36-03:00 - Days: 212.0 - Analysis: 2013-07-30 09:28:36-03:00
283/743 - 15:51:50 Project jblindsay/whitebox-tools - Created: 2018-04-09 19:39:18-03:00 - Days: 212.0 - Analysis: 2018-11-07 19:39:18-03:00
284/743 - 15:51:50 Project jdsjlzx/LRecyclerView - Created: 2016-05-19 07:12:08-03:00 - Days: 212.0 - Analysis: 2016-12-17 07:12:08-03:00
285/743 - 15:51:50 Project jea

...

424/743 - 15:51:51 Project MrVPlusOne/Muse-CGH - Created: 2016-03-02 00:00:07-03:00 - Days: 212.0 - Analysis: 2016-09-30 00:00:07-03:00
425/743 - 15:51:51 Project mtcn/MotionLayoutExamples - Created: 2018-09-06 16:06:51-03:00 - Days: 212.0 - Analysis: 2019-04-06 16:06:51-03:00
426/743 - 15:51:51 Project mtkennerly/ludusavi - Created: 2020-06-20 13:26:46-03:00 - Days: 212.0 - Analysis: 2021-01-18 13:26:46-03:00
427/743 - 15:51:51 Project mtonio91/AMSmoothAlert - Created: 2014-04-24 18:14:27-03:00 - Days: 212.0 - Analysis: 2014-11-22 18:14:27-03:00
428/743 - 15:51:51 Project mukul500/AnimeXStream - Created: 2020-03-13 03:02:52-03:00 - Days: 212.0 - Analysis: 2020-10-11 03:02:52-03:00
429/743 - 15:51:51 Project muzei/muzei - Created: 2014-02-06 20:39:33-03:00 - Days: 212.0 - Analysis: 2014-09-06 20:39:33-03:00
430/743 - 15:51:51 Project mxdldev/android-mvp-mvvm-flytour - Created: 2018-12-25 07:42:31-03:00 - Days: 212.0 - Analysis: 2019-07-25 07:42:31-03:00
431/743 - 15:51:51 Project mytec

496/743 - 15:51:52 Project princewen/tensorflow_practice - Created: 2018-02-23 03:31:20-03:00 - Days: 212.0 - Analysis: 2018-09-23 03:31:20-03:00
497/743 - 15:51:52 Project printfcoder/stack-rpc-tutorials - Created: 2019-04-08 10:28:36-03:00 - Days: 212.0 - Analysis: 2019-11-06 10:28:36-03:00
498/743 - 15:51:52 Project prnicolas/ScalaMl - Created: 2014-06-08 23:38:21-03:00 - Days: 212.0 - Analysis: 2015-01-06 23:38:21-03:00
499/743 - 15:51:52 Project pro100svitlo/FingerprintAuthHelper - Created: 2016-10-06 06:55:58-03:00 - Days: 212.0 - Analysis: 2017-05-06 06:55:58-03:00
500/743 - 15:51:52 Project processwire/processwire - Created: 2016-08-29 06:59:35-03:00 - Days: 212.0 - Analysis: 2017-03-29 06:59:35-03:00
501/743 - 15:51:52 Project programthink/zhao - Created: 2016-02-28 12:38:57-03:00 - Days: 212.0 - Analysis: 2016-09-27 12:38:57-03:00
502/743 - 15:51:52 Project projectlombok/lombok - Created: 2009-06-08 16:46:41-03:00 - Days: 212.0 - Analysis: 2010-01-06 16:46:41-03:00
503/743 - 

562/743 - 15:51:53 Project SmartToolFactory/Animation-Tutorials - Created: 2020-10-02 12:54:15-03:00 - Days: 212.0 - Analysis: 2021-05-02 12:54:15-03:00
563/743 - 15:51:53 Project smashinggit/Study - Created: 2017-12-22 04:44:28-03:00 - Days: 212.0 - Analysis: 2018-07-22 04:44:28-03:00
564/743 - 15:51:53 Project Snailclimb/JavaGuide - Created: 2018-05-07 10:27:00-03:00 - Days: 212.0 - Analysis: 2018-12-05 10:27:00-03:00
565/743 - 15:51:53 Project soapyigu/Swift-30-Projects - Created: 2016-02-14 17:14:21-03:00 - Days: 212.0 - Analysis: 2016-09-13 17:14:21-03:00
566/743 - 15:51:53 Project SoptikHa2/desed - Created: 2020-04-19 21:05:05-03:00 - Days: 212.0 - Analysis: 2020-11-17 21:05:05-03:00
567/743 - 15:51:53 Project spacehuhn/wifi_ducky - Created: 2017-03-12 17:11:27-03:00 - Days: 212.0 - Analysis: 2017-10-10 17:11:27-03:00
568/743 - 15:51:53 Project spatialos/gdk-for-unity - Created: 2018-06-13 12:37:54-03:00 - Days: 212.0 - Analysis: 2019-01-11 12:37:54-03:00
569/743 - 15:51:53 Proje

637/743 - 15:51:53 Project vanhoefm/krackattacks-scripts - Created: 2017-10-18 09:58:08-03:00 - Days: 212.0 - Analysis: 2018-05-18 09:58:08-03:00
638/743 - 15:51:53 Project vaquarkhan/Apache-Kafka-poc-and-notes - Created: 2016-08-05 00:43:14-03:00 - Days: 212.0 - Analysis: 2017-03-05 00:43:14-03:00
639/743 - 15:51:53 Project veracrypt/VeraCrypt - Created: 2014-11-08 19:12:36-03:00 - Days: 212.0 - Analysis: 2015-06-08 19:12:36-03:00
640/743 - 15:51:53 Project vhqtvn/VHEditor-Android - Created: 2020-05-24 16:38:00-03:00 - Days: 212.0 - Analysis: 2020-12-22 16:38:00-03:00
641/743 - 15:51:53 Project vicc/chameleon - Created: 2014-07-26 06:01:09-03:00 - Days: 212.0 - Analysis: 2015-02-23 06:01:09-03:00
642/743 - 15:51:53 Project vicedev/ZY-Player-Android - Created: 2020-06-09 04:41:27-03:00 - Days: 212.0 - Analysis: 2021-01-07 04:41:27-03:00
643/743 - 15:51:53 Project VideoFlint/Cabbage - Created: 2018-07-31 22:42:02-03:00 - Days: 212.0 - Analysis: 2019-02-28 22:42:02-03:00
644/743 - 15:51:

702/743 - 15:51:54 Project yipianfengye/android-zxingLibrary - Created: 2016-07-26 05:00:18-03:00 - Days: 212.0 - Analysis: 2017-02-23 05:00:18-03:00
703/743 - 15:51:54 Project ymcui/Chinese-BERT-wwm - Created: 2019-06-19 19:55:01-03:00 - Days: 212.0 - Analysis: 2020-01-17 19:55:01-03:00
704/743 - 15:51:54 Project YMFE/yapi - Created: 2016-05-30 06:09:09-03:00 - Days: 212.0 - Analysis: 2016-12-28 06:09:09-03:00
705/743 - 15:51:54 Project youngsoft/MyLinearLayout - Created: 2015-06-13 14:26:16-03:00 - Days: 212.0 - Analysis: 2016-01-11 14:26:16-03:00
706/743 - 15:51:54 Project youth5201314/banner - Created: 2016-04-22 02:50:25-03:00 - Days: 212.0 - Analysis: 2016-11-20 02:50:25-03:00
707/743 - 15:51:54 Project youusername/magnetX - Created: 2016-10-20 07:23:11-03:00 - Days: 212.0 - Analysis: 2017-05-20 07:23:11-03:00
708/743 - 15:51:54 Project YouXianMing/Animations - Created: 2015-11-16 09:25:29-03:00 - Days: 212.0 - Analysis: 2016-06-15 09:25:29-03:00
709/743 - 15:51:54 Project ysrc/w

##### 1.4. Setting Analysis_init and Analysis_finish

In [100]:
repos = getProjectsMined()
i=0

for repo in repos:    
    project = repo[0]
    first_commit = repo[1]
    
    #Calculate init of the moth and finish of the last month
    #Adjust the finish date to the first hour of the subsequent month
    analysisInit = analysisFinish = datetime(first_commit.year, first_commit.month, 1, 0,0,0,tzinfo=first_commit.tzinfo)
    analysisFinish = analysisFinish + relativedelta(months=+13)

    
    setAnalysis_init_finish(project,analysisInit,analysisFinish)
        
    
    i += 1
    print('{}/{} - Project {} - Init: {} - Finish: {}'.format(i,len(repos),project,analysisInit,analysisFinish))
    #time.sleep(.02)
        

1/876 - Project zzzeek/sqlalchemy - Init: 2013-12-01 00:00:00-03:00 - Finish: 2015-01-01 00:00:00-03:00
2/876 - Project zzanehip/The-OldOS-Project - Init: 2022-01-01 00:00:00-03:00 - Finish: 2023-02-01 00:00:00-03:00
3/876 - Project zyprosoft/ZYChat - Init: 2016-02-01 00:00:00-03:00 - Finish: 2017-03-01 00:00:00-03:00
4/876 - Project zuoqing1988/ZQCNN - Init: 2018-12-01 00:00:00-03:00 - Finish: 2020-01-01 00:00:00-03:00
5/876 - Project zuihou/lamp-cloud - Init: 2018-08-01 00:00:00-03:00 - Finish: 2019-09-01 00:00:00-03:00
6/876 - Project zouzg/mybatis-generator-gui - Init: 2016-12-01 00:00:00-03:00 - Finish: 2018-01-01 00:00:00-03:00
7/876 - Project zlt2000/microservices-platform - Init: 2019-10-01 00:00:00-03:00 - Finish: 2020-11-01 00:00:00-03:00
8/876 - Project zixun/CocoaChinaPlus - Init: 2016-05-01 00:00:00-03:00 - Finish: 2017-06-01 00:00:00-03:00
9/876 - Project zipme/RQShineLabel - Init: 2014-12-01 00:00:00-03:00 - Finish: 2016-01-01 00:00:00-03:00
10/876 - Project ZiniuLu/Pyth

77/876 - Project wszqkzqk/deepin-wine-ubuntu - Init: 2019-01-01 00:00:00-03:00 - Finish: 2020-02-01 00:00:00-03:00
78/876 - Project wsl2ls/iOS_Tips - Init: 2020-04-01 00:00:00-03:00 - Finish: 2021-05-01 00:00:00-03:00
79/876 - Project worldveil/dejavu - Init: 2014-06-01 00:00:00-03:00 - Finish: 2015-07-01 00:00:00-03:00
80/876 - Project wordpress-mobile/WordPress-iOS - Init: 2016-05-01 00:00:00-03:00 - Finish: 2017-06-01 00:00:00-03:00
81/876 - Project woltapp/blurhash - Init: 2020-01-01 00:00:00-03:00 - Finish: 2021-02-01 00:00:00-03:00
82/876 - Project wistbean/learn_python3_spider - Init: 2019-10-01 00:00:00-03:00 - Finish: 2020-11-01 00:00:00-03:00
83/876 - Project wine-mirror/wine - Init: 2015-03-01 00:00:00-03:00 - Finish: 2016-04-01 00:00:00-03:00
84/876 - Project Wind4/vlmcsd - Init: 2016-06-01 00:00:00-03:00 - Finish: 2017-07-01 00:00:00-03:00
85/876 - Project wildfirechat/ios-chat - Init: 2019-08-01 00:00:00-03:00 - Finish: 2020-09-01 00:00:00-03:00
86/876 - Project wigl/iSim

167/876 - Project SystemErrorWang/White-box-Cartoonization - Init: 2020-10-01 00:00:00-03:00 - Finish: 2021-11-01 00:00:00-03:00
168/876 - Project symplify/symplify - Init: 2020-01-01 00:00:00-03:00 - Finish: 2021-02-01 00:00:00-03:00
169/876 - Project SwooleDistributed/SwooleDistributed - Init: 2017-02-01 00:00:00-03:00 - Finish: 2018-03-01 00:00:00-03:00
170/876 - Project SwiftKickMobile/SwiftMessages - Init: 2017-03-01 00:00:00-03:00 - Finish: 2018-04-01 00:00:00-03:00
171/876 - Project Swift-AI/Swift-AI - Init: 2016-06-01 00:00:00-03:00 - Finish: 2017-07-01 00:00:00-03:00
172/876 - Project SusionSuc/AdvancedAndroid - Init: 2019-05-01 00:00:00-03:00 - Finish: 2020-06-01 00:00:00-03:00
173/876 - Project surge-synthesizer/surge - Init: 2019-04-01 00:00:00-03:00 - Finish: 2020-05-01 00:00:00-03:00
174/876 - Project SUPERAndroidAnalyzer/super - Init: 2016-06-01 00:00:00-03:00 - Finish: 2017-07-01 00:00:00-03:00
175/876 - Project SunshineBrother/JHBlog - Init: 2019-02-01 00:00:00-03:00 -

245/876 - Project RocketsChen/CDDStore - Init: 2017-10-01 00:00:00-03:00 - Finish: 2018-11-01 00:00:00-03:00
246/876 - Project roberthein/BouncyLayout - Init: 2017-11-01 00:00:00-03:00 - Finish: 2018-12-01 00:00:00-03:00
247/876 - Project roberdam/Xaddress - Init: 2017-03-01 00:00:00-03:00 - Finish: 2018-04-01 00:00:00-03:00
248/876 - Project riy/degraph - Init: 2012-03-01 00:00:00-03:00 - Finish: 2013-04-01 00:00:00-03:00
249/876 - Project rileytestut/AltStore - Init: 2019-12-01 00:00:00-03:00 - Finish: 2021-01-01 00:00:00-03:00
250/876 - Project richmondu/libfaceid - Init: 2019-06-01 00:00:00-03:00 - Finish: 2020-07-01 00:00:00-03:00
251/876 - Project revely-inc/co.revely.gradient - Init: 2018-04-01 00:00:00-03:00 - Finish: 2019-05-01 00:00:00-03:00
252/876 - Project RetroMusicPlayer/RetroMusicPlayer - Init: 2019-02-01 00:00:00-03:00 - Finish: 2020-03-01 00:00:00-03:00
253/876 - Project Renovamen/playground-macos - Init: 2021-11-01 00:00:00-03:00 - Finish: 2022-12-01 00:00:00-03:00
2

321/876 - Project opensourceteams/spark-scala-maven-2.4.0 - Init: 2019-07-01 00:00:00-03:00 - Finish: 2020-08-01 00:00:00-03:00
322/876 - Project open-sdr/openwifi - Init: 2020-06-01 00:00:00-03:00 - Finish: 2021-07-01 00:00:00-03:00
323/876 - Project openmediavault/openmediavault - Init: 2016-12-01 00:00:00-03:00 - Finish: 2018-01-01 00:00:00-03:00
324/876 - Project openfarmcc/OpenFarm - Init: 2020-02-01 00:00:00-03:00 - Finish: 2021-03-01 00:00:00-03:00
325/876 - Project omz/AppSales-Mobile - Init: 2010-01-01 00:00:00-03:00 - Finish: 2011-02-01 00:00:00-03:00
326/876 - Project OmGodse/Notally - Init: 2020-12-01 00:00:00-03:00 - Finish: 2022-01-01 00:00:00-03:00
327/876 - Project omarsahl/Flopsy - Init: 2019-01-01 00:00:00-03:00 - Finish: 2020-02-01 00:00:00-03:00
328/876 - Project olikraus/u8glib - Init: 2016-01-01 00:00:00-03:00 - Finish: 2017-02-01 00:00:00-03:00
329/876 - Project olikraus/u8g2 - Init: 2016-06-01 00:00:00-03:00 - Finish: 2017-07-01 00:00:00-03:00
330/876 - Project 

...

487/876 - Project lcdevelop/ChatBotCourse - Init: 2017-01-01 00:00:00-03:00 - Finish: 2018-02-01 00:00:00-03:00
488/876 - Project lattera/glibc - Init: 2012-10-01 00:00:00-03:00 - Finish: 2013-11-01 00:00:00-03:00
489/876 - Project Latitude-Archives/AIDungeon - Init: 2019-10-01 00:00:00-03:00 - Finish: 2020-11-01 00:00:00-03:00
490/876 - Project larrynatalicio/15DaysofAnimationsinSwift - Init: 2016-12-01 00:00:00-03:00 - Finish: 2018-01-01 00:00:00-03:00
491/876 - Project ladybirdweb/faveo-helpdesk - Init: 2019-05-01 00:00:00-03:00 - Finish: 2020-06-01 00:00:00-03:00
492/876 - Project labstack/armor - Init: 2016-10-01 00:00:00-03:00 - Finish: 2017-11-01 00:00:00-03:00
493/876 - Project kythe/kythe - Init: 2015-08-01 00:00:00-03:00 - Finish: 2016-09-01 00:00:00-03:00
494/876 - Project kyleduo/TinyPNG4Mac - Init: 2017-01-01 00:00:00-03:00 - Finish: 2018-02-01 00:00:00-03:00
495/876 - Project kyleduo/SwitchButton - Init: 2015-04-01 00:00:00-03:00 - Finish: 2016-05-01 00:00:00-03:00
496/87

567/876 - Project i18next/i18next - Init: 2016-01-01 00:00:00-03:00 - Finish: 2017-02-01 00:00:00-03:00
568/876 - Project Hydrospheredata/mist - Init: 2017-10-01 00:00:00-03:00 - Finish: 2018-11-01 00:00:00-03:00
569/876 - Project hugeterry/CoordinatorTabLayout - Init: 2017-08-01 00:00:00-03:00 - Finish: 2018-09-01 00:00:00-03:00
570/876 - Project hugapi/hug - Init: 2019-05-01 00:00:00-03:00 - Finish: 2020-06-01 00:00:00-03:00
571/876 - Project huangyanbin/smartTable - Init: 2018-06-01 00:00:00-03:00 - Finish: 2019-07-01 00:00:00-03:00
572/876 - Project huanghaibin-dev/CalendarView - Init: 2017-10-01 00:00:00-03:00 - Finish: 2018-11-01 00:00:00-03:00
573/876 - Project huangfangyi/YiChat - Init: 2016-08-01 00:00:00-03:00 - Finish: 2017-09-01 00:00:00-03:00
574/876 - Project huang303513/Design-Pattern-For-iOS - Init: 2016-03-01 00:00:00-03:00 - Finish: 2017-04-01 00:00:00-03:00
575/876 - Project hs-web/hsweb-framework - Init: 2017-10-01 00:00:00-03:00 - Finish: 2018-11-01 00:00:00-03:00


648/876 - Project fighting41love/funNLP - Init: 2019-03-01 00:00:00-03:00 - Finish: 2020-04-01 00:00:00-03:00
649/876 - Project fidloo/flux - Init: 2021-10-01 00:00:00-03:00 - Finish: 2022-11-01 00:00:00-03:00
650/876 - Project ferPrieto/Coroutines-Flows-Modularised - Init: 2020-07-01 00:00:00-03:00 - Finish: 2021-08-01 00:00:00-03:00
651/876 - Project fengjundev/Android-Skin-Loader - Init: 2016-01-01 00:00:00-03:00 - Finish: 2017-02-01 00:00:00-03:00
652/876 - Project feiyangqingyun/QWidgetDemo - Init: 2020-05-01 00:00:00-03:00 - Finish: 2021-06-01 00:00:00-03:00
653/876 - Project feelschaotic/SDKHotFix - Init: 2020-02-01 00:00:00-03:00 - Finish: 2021-03-01 00:00:00-03:00
654/876 - Project fechanique/cordova-plugin-fcm - Init: 2017-01-01 00:00:00-03:00 - Finish: 2018-02-01 00:00:00-03:00
655/876 - Project FabrizioBrancati/BFKit-Swift - Init: 2018-08-01 00:00:00-03:00 - Finish: 2019-09-01 00:00:00-03:00
656/876 - Project F5OEO/rpitx - Init: 2016-05-01 00:00:00-03:00 - Finish: 2017-06-0

728/876 - Project czl0325/ZLCollectionView - Init: 2018-01-01 00:00:00-03:00 - Finish: 2019-02-01 00:00:00-03:00
729/876 - Project cysmith/neural-style-tf - Init: 2017-05-01 00:00:00-03:00 - Finish: 2018-06-01 00:00:00-03:00
730/876 - Project cx9208/bbrplus - Init: 2019-07-01 00:00:00-03:00 - Finish: 2020-08-01 00:00:00-03:00
731/876 - Project cundong/SmartAppUpdates - Init: 2014-07-01 00:00:00-03:00 - Finish: 2015-08-01 00:00:00-03:00
732/876 - Project Cuberto/bubble-icon-tabbar - Init: 2019-09-01 00:00:00-03:00 - Finish: 2020-10-01 00:00:00-03:00
733/876 - Project CTurt/FreeDVDBoot - Init: 2021-01-01 00:00:00-03:00 - Finish: 2022-02-01 00:00:00-03:00
734/876 - Project crossoverJie/cim - Init: 2018-12-01 00:00:00-03:00 - Finish: 2020-01-01 00:00:00-03:00
735/876 - Project croogo/croogo - Init: 2017-09-01 00:00:00-03:00 - Finish: 2018-10-01 00:00:00-03:00
736/876 - Project coyove/goflyway - Init: 2018-03-01 00:00:00-03:00 - Finish: 2019-04-01 00:00:00-03:00
737/876 - Project CouchPotat

820/876 - Project aslanyanhaik/Quick-Chat - Init: 2017-08-01 00:00:00-03:00 - Finish: 2018-09-01 00:00:00-03:00
821/876 - Project ashleymcnamara/gophers - Init: 2017-09-01 00:00:00-03:00 - Finish: 2018-10-01 00:00:00-03:00
822/876 - Project Ascotbe/Kernelhub - Init: 2021-03-01 00:00:00-03:00 - Finish: 2022-04-01 00:00:00-03:00
823/876 - Project Asabeneh/30-Days-Of-React - Init: 2021-04-01 00:00:00-03:00 - Finish: 2022-05-01 00:00:00-03:00
824/876 - Project Asabeneh/30-Days-Of-Python - Init: 2020-06-01 00:00:00-03:00 - Finish: 2021-07-01 00:00:00-03:00
825/876 - Project ARMmbed/mbed-os - Init: 2020-09-01 00:00:00-03:00 - Finish: 2021-10-01 00:00:00-03:00
826/876 - Project armcha/ColoredShadowImageView - Init: 2018-09-01 00:00:00-03:00 - Finish: 2019-10-01 00:00:00-03:00
827/876 - Project armcha/AutoLinkTextViewV2 - Init: 2020-04-01 00:00:00-03:00 - Finish: 2021-05-01 00:00:00-03:00
828/876 - Project arlicle/panda-api - Init: 2020-08-01 00:00:00-03:00 - Finish: 2021-09-01 00:00:00-03:00


##### 1.5. (Re)Process metrics quantities in analysis period:
- Notebook 3_1:
    - analysis_releases
    - analysis_issues
    - analysis_prs
- Notebook 5_1:
    - Mine Communication of pull requests in selected releases
- Notebook 6:
    - Consolidate pullrequests data via Github API
- Notebook 7:
    - qty_bugs_period
    - qty_bugs
    - Merge_conflicts

## Metrics for CI classification
* Build Health (Median)
    * Rate of builds passed.
* Builds activity (Median)
    * Frequency of days having builds in a given period of time (e.g. 10/30 days having builds in a month).
* Time to Fix broken builds (Median)
    * Time since a build fail status until the next success.
* Test Coverage (Median)
    * Percent of test coverage in a build

### Process
- Get analysis period (projects.analysis_init - projects.analysis_finish) for each project:
    - Calculate Build Health, Build activity (integration frequency), Time to Fix broken builds, and Test Coverage;
    - Get projects having values for coverage and build greater than 0;

#### Calculate and record metric fields on table Projects

In [44]:
repos = getProjects()

for repo in repos:    
    project = repo[0]
    analysisInit = repo[1]
    analysisFinish = repo[2]
    print('Processing CI Metrics. Project:\t {}\t {}  -  {}'.format(project,analysisInit,analysisFinish))
    
    buildHealth = calcBuildHealth(project,analysisInit,analysisFinish)
    timeToFix = calcTimeToFix(project,analysisInit,analysisFinish)
    builds_activity = calcActivity(project,analysisInit,analysisFinish)
    coverage = calcCoverage(project,analysisInit,analysisFinish)
    
    updateProjectMetrics(project,buildHealth, timeToFix, builds_activity,coverage)

Processing CI Metrics. Project:	 agershun/alasql	 2017-12-01 00:00:00-03:00  -  2019-01-01 00:00:00-03:00
			 Build Health: 0.13059701492537312 - Total builds: 268
			 Time To Fix: 777895.5 - Qty data ttf: 232
			 Build Activity: 0.5378787878787878 - Total Activity Days: 213 - Total Days: 396
			 Coverage: 90.40794 - Qty data cov: 2
Processing CI Metrics. Project:	 airbnb/visx	 2020-09-01 00:00:00-03:00  -  2021-10-01 00:00:00-03:00
			 Build Health: 0 - Total builds: 0
			 Time To Fix: 0 - Qty data ttf: 0
			 Build Activity: 0.0 - Total Activity Days: 0 - Total Days: 395
			 Coverage: 60.0952 - Qty data cov: 649
Processing CI Metrics. Project:	 amark/gun	 2018-01-01 00:00:00-03:00  -  2019-02-01 00:00:00-03:00
			 Build Health: 0.04823151125401929 - Total builds: 311
			 Time To Fix: 3685336.0 - Qty data ttf: 290
			 Build Activity: 0.30808080808080807 - Total Activity Days: 122 - Total Days: 396
			 Coverage: 53.3951005 - Qty data cov: 2
Processing CI Metrics. Project:	 amphp/http-se

			 Time To Fix: 1709099.0 - Qty data ttf: 137
			 Build Activity: 0.2601010101010101 - Total Activity Days: 103 - Total Days: 396
			 Coverage: 29.004372 - Qty data cov: 113
Processing CI Metrics. Project:	 errbit/errbit	 2020-02-01 00:00:00-03:00  -  2021-03-01 00:00:00-03:00
			 Build Health: 0.040955631399317405 - Total builds: 293
			 Time To Fix: 2424037.0 - Qty data ttf: 279
			 Build Activity: 0.649746192893401 - Total Activity Days: 256 - Total Days: 394
			 Coverage: 93.441765 - Qty data cov: 312
Processing CI Metrics. Project:	 ethereum/go-ethereum	 2020-09-01 00:00:00-03:00  -  2021-10-01 00:00:00-03:00
			 Build Health: 0.37254901960784315 - Total builds: 51
			 Time To Fix: 3940.0 - Qty data ttf: 20
			 Build Activity: 0.017721518987341773 - Total Activity Days: 7 - Total Days: 395
			 Coverage: 36.303734 - Qty data cov: 3
Processing CI Metrics. Project:	 EyalAr/lwip	 2014-09-01 00:00:00-03:00  -  2015-10-01 00:00:00-03:00
			 Build Health: 0.14919354838709678 - Total bui

			 Build Activity: 0.20906801007556675 - Total Activity Days: 83 - Total Days: 397
			 Coverage: nan - Qty data cov: 0
Processing CI Metrics. Project:	 michael-lazar/rtv	 2018-02-01 00:00:00-03:00  -  2019-03-01 00:00:00-03:00
			 Build Health: 0.20218579234972678 - Total builds: 183
			 Time To Fix: 899923.0 - Qty data ttf: 146
			 Build Activity: 0.16539440203562342 - Total Activity Days: 65 - Total Days: 393
			 Coverage: 92.88844 - Qty data cov: 155
Processing CI Metrics. Project:	 microsoft/msphpsql	 2019-04-01 00:00:00-03:00  -  2020-05-01 00:00:00-03:00
			 Build Health: 0.1527777777777778 - Total builds: 288
			 Time To Fix: 643661.0 - Qty data ttf: 244
			 Build Activity: 0.29292929292929293 - Total Activity Days: 116 - Total Days: 396
			 Coverage: 73.665016 - Qty data cov: 306
Processing CI Metrics. Project:	 minishift/minishift	 2019-04-01 00:00:00-03:00  -  2020-05-01 00:00:00-03:00
			 Build Health: 0.0875 - Total builds: 160


/home/eliezio/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


			 Time To Fix: 520741.0 - Qty data ttf: 143
			 Build Activity: 0.12121212121212122 - Total Activity Days: 48 - Total Days: 396
			 Coverage: 71.662159 - Qty data cov: 2
Processing CI Metrics. Project:	 missionpinball/mpf	 2021-03-01 00:00:00-03:00  -  2022-04-01 00:00:00-03:00
			 Build Health: 0.22872340425531915 - Total builds: 188
			 Time To Fix: 326045.5 - Qty data ttf: 120
			 Build Activity: 0.10353535353535354 - Total Activity Days: 41 - Total Days: 396
			 Coverage: 0 - Qty data cov: 0
Processing CI Metrics. Project:	 nats-io/nats-server	 2020-06-01 00:00:00-03:00  -  2021-07-01 00:00:00-03:00
			 Build Health: 0.17174515235457063 - Total builds: 361
			 Time To Fix: 35356.0 - Qty data ttf: 258
			 Build Activity: 0.10886075949367088 - Total Activity Days: 43 - Total Days: 395
			 Coverage: 87.87644499999999 - Qty data cov: 1898
Processing CI Metrics. Project:	 nats-io/nats-streaming-server	 2019-08-01 00:00:00-03:00  -  2020-09-01 00:00:00-03:00
			 Build Health: 0.0504201

			 Build Health: 0.5496688741721855 - Total builds: 151
			 Time To Fix: 836322.0 - Qty data ttf: 67
			 Build Activity: 0.20202020202020202 - Total Activity Days: 80 - Total Days: 396
			 Coverage: 87.79928 - Qty data cov: 3
Processing CI Metrics. Project:	 Ullaakut/cameradar	 2018-02-01 00:00:00-03:00  -  2019-03-01 00:00:00-03:00
			 Build Health: 0.058394160583941604 - Total builds: 137
			 Time To Fix: 4053128.0 - Qty data ttf: 109
			 Build Activity: 0.061068702290076333 - Total Activity Days: 24 - Total Days: 393
			 Coverage: 96.65738 - Qty data cov: 73
Processing CI Metrics. Project:	 variadico/noti	 2016-04-01 00:00:00-03:00  -  2017-05-01 00:00:00-03:00
			 Build Health: 0.5039370078740157 - Total builds: 127
			 Time To Fix: 1281772.0 - Qty data ttf: 63
			 Build Activity: 0.08860759493670886 - Total Activity Days: 35 - Total Days: 395
			 Coverage: 53.721684 - Qty data cov: 13
Processing CI Metrics. Project:	 wallix/awless	 2018-06-01 00:00:00-03:00  -  2019-07-01 00:00:0

#### Flag CI Projects
- Projects having:
    - coverage > 0;
    - Build (activity, health, ttf) > 0;
    - prs_selected and issues_selected
    - Travis CI project
    
#### Select CI Projects
- Projects having:
    - CI flag true (previous bullet);
    - Bugs in analysis period;
    
#### Draw the same quantity of projects through NO CI instances
- Projects having:
    - No CI server
    - prs_selected and issues_selected
    - Bugs in analysis period


In [45]:
updateCIProjects()

qty_projects = getQtyProjects()

# Which of these groups is the lower?
# Select all projects from this lower group
# Then, draw the same quantity from the other.
if qty_projects[0][1] < qty_projects[1][1]:
    setSelectedAllProjects(qty_projects[0][0])
    drawProjects(qty_projects[1][0],qty_projects[0][1])
else:
    setSelectedAllProjects(qty_projects[1][0])
    drawProjects(qty_projects[0][0],qty_projects[1][1])
    
#drawNOCIProjects(1)

PROJECT: docker-archive/deploykit
PROJECT: kofigumbs/multi
PROJECT: nenosllc/iCloudDocumentSync
PROJECT: li-xiaojun/XPopup
PROJECT: cobub/razor
PROJECT: romandanylyk/PageIndicatorView
PROJECT: coyove/goflyway
PROJECT: Marxon13/M13ProgressSuite
PROJECT: V2Ray-Android/Actinium
PROJECT: bytefury/crater
PROJECT: Mortennn/Dozer
PROJECT: BijoySingh/Scarlet-Notes
PROJECT: ivanvorobei/SPAlert
PROJECT: TruthHun/DocHub
PROJECT: veracrypt/VeraCrypt
PROJECT: react-native-video/react-native-video
PROJECT: wazuh/wazuh
PROJECT: hanjoongcho/aaf-easydiary
PROJECT: mazurio/bodyweight-fitness-android
PROJECT: googleforgames/open-match
PROJECT: crossoverJie/cim
PROJECT: sedwards2009/extraterm
PROJECT: HMBSbige/ShadowsocksR-Android
PROJECT: saket/InboxRecyclerView
PROJECT: spatialos/gdk-for-unity
PROJECT: jdsjlzx/LRecyclerView
PROJECT: Lora-net/LoRaMac-node
PROJECT: jblindsay/whitebox-tools
PROJECT: HDB-Li/LLDebugTool
PROJECT: material-shell/material-shell
PROJECT: sixtyfpsui/sixtyfps
PROJECT: davideas/Fle

In [2]:
def drawProjects(ci,qty):
    projects = getGroupProjects(ci)
    projects = list(map(lambda x: x[0], projects))
    i=0
    
    while i < qty:
        random.shuffle(projects)
        id = random.randint(0, len(projects)-1)
        proj = projects.pop(id)
        
        i +=1
        
        print('PROJECT: {}'.format(proj))
        try:
            query = """UPDATE  projects
                set rq1_included = true
                WHERE repo_name like %s"""

            connection = connectDB()
            cursor = connection.cursor()
            cursor.execute(query,[proj])
            connection.commit()
            cursor.close()
            connection.close()
        except psycopg2.IntegrityError as e:
            print ("==============================================================")
            print ("Error while updating into PostgreSQL. drawProjects >>> Exception: {}".format(e)) 
            connection.close()
        except Exception as e:
            print ("==============================================================")
            print ("Error while processing drawProjects >>> Exception: {}".format(e)) 

In [3]:
def getGroupProjects(ci):
    query = """SELECT repo_name From projects 
                WHERE analysis_issues > 0 AND analysis_prs > 0 AND commits_mined is true 
                AND analysis_releases > 0 AND qty_bugs > 0"""
    
    if ci:
        query += """ AND builds_activity > 0 AND coverage > 0 AND ci_service is not null;"""
    else:
        query += """ AND ci_service is NULL;"""
            

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [4]:
def setSelectedAllProjects(ci):
    try:
        query = """UPDATE  projects
            set rq1_included = true
            WHERE analysis_issues > 0 AND analysis_prs > 0 AND commits_mined is true 
                    AND analysis_releases > 0 AND qty_bugs > 0"""
        
        if ci:
            query += """ AND builds_activity > 0 AND coverage > 0 AND ci_service is not null;"""
        else:
            query += """ AND ci_service is null;"""
            

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. setSelectedAllProjects >>> Exception: {}".format(e)) 
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing setSelectedAllProjects >>> Exception: {}".format(e)) 

In [5]:
def getQtyProjects():
    query = """select ci,count(repo_name) from projects WHERE 
                    analysis_issues > 0 AND analysis_prs > 0 AND commits_mined is true AND analysis_releases > 0 AND qty_bugs > 0 AND 
                    ((builds_activity > 0 AND coverage > 0 AND ci_service is not null) 
                    OR ci_service is null)
                group by ci;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [6]:
def updateCIProjects():
    try:
        query = """UPDATE  projects 
            set ci = true
            WHERE repo_name IN
                (select repo_name from projects WHERE 
                    (commits_mined IS TRUE and analysis_releases > 0 and analysis_issues > 0 and analysis_prs > 0 
                    and qty_bugs > 0
                    and analysis_point_days is not null)
                AND CI_SERVICE iLIKE 'TRAVIS CI'
                AND coverage > 0 AND builds_activity > 0);                
            """

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. updateCIProjects >>> Exception: {}".format(e)) 
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing updateCIProjects >>> Exception: {}".format(e)) 

In [7]:
def drawNOCIProjects(qty):
    projects = getNOCIProjects()
    projects = list(map(lambda x: x[0], projects))
    i=0
    
    while i < qty:
        random.shuffle(projects)
        id = random.randint(0, len(projects)-1)
        proj = projects.pop(id)
        
        i +=1
        
        print('PROJECT: {}'.format(proj))
        try:
            query = """UPDATE  projects
                set rq1 = true
                WHERE repo_name like %s"""

            connection = connectDB()
            cursor = connection.cursor()
            cursor.execute(query,[proj])
            connection.commit()
            cursor.close()
            connection.close()
        except psycopg2.IntegrityError as e:
            print ("==============================================================")
            print ("Error while updating into PostgreSQL. drawNOCIProjects >>> Exception: {}".format(e)) 
            connection.close()
        except Exception as e:
            print ("==============================================================")
            print ("Error while processing drawNOCIProjects >>> Exception: {}".format(e)) 

In [8]:
#calcActivity('3b1b/manim','2019-10-01 00:00:00-03:00','2020-10-01 00:00:00-03:00')
#r=getBuildDays('3b1b/manim','2019-10-01 00:00:00-03','2020-10-01 00:00:00-03')
#len(r)

#r=getCoverage('ruslanskorb/RSKImageCropper','2015-03-01 00:00:00-03:00','2016-03-01 00:00:00-03:00')
#type(r)
#if r.empty:
#    print('eita')
#calcCoverage('3b1b/manim','2019-10-01 00:00:00-03','2020-10-01 00:00:00-03')
#r
#print(r['coverage'].median())
#r = removeOutliers(r)
#print(r['coverage'].median())
#failBuilds = getFailBuilds('3b1b/manim')
#nextDate = getNextSuccess('3b1b/manim',failBuilds[0][1])
#nextDate[0][1]
#diff = (nextDate[0][1] - failBuilds[0][1]).total_seconds()
#diff
#fillTimeToNextSuccess('3b1b/manim')

#calcTimeToFix('3b1b/manim','2019-10-01 00:00:00-03','2020-10-01 00:00:00-03')
#ttfs = getTTFs('3b1b/manim','2019-10-01 00:00:00-03','2020-10-01 00:00:00-03')
#ttfs['time_to_fix'].median()

### Aux Functions

In [9]:
import psycopg2
import os
import requests 
import time
import pytz    
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import dateutil.parser
import random
import pandas as pd
from scipy import stats
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

In [10]:
def connectDB():
    f = open('/home/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    return psycopg2.connect(user = "ci_quality",
                              password = pwd,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "Causal_CI_Quality_v4")

In [11]:
def connectDBPandas():
    f = open('/home/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    #"User ID=ci_quality;Password={};Host=localhost;Port=5432;Database=Causal_CI_Quality_v4;Pooling=true;Min Pool Size=0;Max Pool Size=100;Connection Lifetime=0;".format(pwd)
    DATABASE_URI = 'postgresql://ci_quality:{}@localhost:5432/Causal_CI_Quality_v4'.format(pwd)
    #connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
    
    return create_engine(DATABASE_URI)

In [12]:
def getProjects():
    #query = """SELECT repo_name, min(init_period), max(end_period) From metrics_period 
    #    WHERE period ilike 'month' and repo_name IN 
    #        (SELECT repo_name From PROJECTS WHERE (prs_selected IS TRUE and issues_selected is true) AND CI_SERVICE iLIKE 'TRAVIS CI')
    #    GROUP BY repo_name
    #    order by repo_name --offset 400"""

    query = """SELECT repo_name, analysis_init, analysis_finish, analysis_point From PROJECTS 
    WHERE commits_mined IS TRUE and analysis_releases > 0 and analysis_issues > 0 and analysis_prs > 0 
    and qty_bugs > 0
    and analysis_point_days is not null
    and CI_SERVICE ILIKE 'TRAVIS CI'
    order by repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [13]:
def getProjectsToRecalculate():
    query = """SELECT repo_name, created From PROJECTS 
    WHERE CI_SERVICE ILIKE 'TRAVIS CI'
    order by repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [14]:
def getNOCIProjects():
    query = """SELECT repo_name From PROJECTS 
    WHERE commits_mined2 IS TRUE and analysis_releases > 1 and analysis_issues > 0 and analysis_prs > 0 
    and qty_bugs_period > 0
    and CI_SERVICE is NULL
    order by repo_name;"""
    

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [15]:
def getAllProjects():
    query = """SELECT repo_name From PROJECTS 
    WHERE commits_mined2 IS TRUE and analysis_releases > 1 and analysis_issues > 0 and analysis_prs > 0 
    and qty_bugs_period > 0
    order by repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [16]:
def calcBuildHealth(project,init, finish):
    builds = getBuildResults(project, init, finish)
    buildHealth = computeBuildHealth(builds)

    print('\t\t\t Build Health: {} - Total builds: {}'.format(buildHealth,len(builds)))
    return buildHealth

In [17]:
def calcTimeToFix(project,init, finish):
    fillTimeToNextSuccess(project)
    
    ttfs = getTTFs(project, init, finish)
    if ttfs.empty:
        ttf = 0
    else:
        ttf = ttfs['time_to_fix'].median()
        
    print('\t\t\t Time To Fix: {} - Qty data ttf: {}'.format(ttf,ttfs['time_to_fix'].count()))
    return ttf

In [18]:
def fillTimeToNextSuccess(project):
    #Get all failed builds
    failBuilds = getFailBuilds(project)
    #For each build, get the next success and update it.
    for b in failBuilds:
        fail = b[1]
        nextDate = getNextSuccess(project,fail)#created
        if nextDate is not None and len(nextDate) > 0:
            success = nextDate[0][1]
        
            diff = (success - fail).total_seconds()
        
            updateBuildTTF(project,b[0],diff)

In [19]:
def calcActivity(project,init, finish):
    #init = dateutil.parser.parse(init)
    #finish = dateutil.parser.parse(finish)
    days = (finish - init).days
    
    build_days = getBuildDays(project, init, finish)
    build_activity = len(build_days)/days

    print('\t\t\t Build Activity: {} - Total Activity Days: {} - Total Days: {}'.format(build_activity,len(build_days),days))
    return build_activity

In [20]:
def calcCoverage(project,init, finish):
    covs = getCoverage(project, init, finish)
    if covs.empty:
        coverage = 0
    else:
        if checkValues(covs['coverage']):
            coverage = covs['coverage'].median()
        else:
           # coverage = removeOutliers(covs)
            coverage = covs['coverage'].median()
        
        #coverage = covs['coverage'].median()
        
    print('\t\t\t Coverage: {} - Qty data cov: {}'.format(coverage,covs['coverage'].count()))
    return coverage

In [21]:
def getBuildResults(repo_name,initDate, finishDate):    
    query = """SELECT result FROM builds_mined WHERE 
            repo_name = %s AND ((STARTED_at BETWEEN %s AND %s) OR (finishED_at BETWEEN %s AND %s))"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name,initDate, finishDate,initDate, finishDate])
    rows = cursor.fetchall()
    
    
    cursor.close()
    connection.close()
    return rows

In [22]:
def getFailBuilds(repo_name):    
    query = """select build_number, started_at, result from builds_mined 
        WHERE repo_name = %s AND result is False AND started_at is not NULL AND time_to_fix is NULL
        order by started_at"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    
    cursor.close()
    connection.close()
    return rows

In [23]:
def getNextSuccess(repo_name,date):    
    query = """select build_number, started_at, result from builds_mined 
        WHERE repo_name = %s AND result is True  AND started_at > %s
        order by started_at limit 1;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name,date])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [24]:
def getBuildDays(project, init, finish):
    query = """select distinct date(started_at) from builds_mined
                WHERE repo_name = %s AND (STARTED_at BETWEEN %s AND %s);"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [project,init, finish])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [25]:
def getCoverage(project_name,init,finish):
    connection = connectDBPandas()
    query = 'select coverage from coverage where (created BETWEEN %s AND %s) AND repo_name like %s;'

    df = pd.read_sql_query(query,con=connection,params=[init,finish,project_name])
    #connection.close()
    
    #Check if column contains all values 0. In this case we do not filter outliers.
    #if checkValues(df['coverage']):
    return df
    #else:
    #    return removeOutliers(df)

In [26]:
def getTTFs(project_name,init,finish):
    connection = connectDBPandas()
    query = 'select time_to_fix from builds_mined where time_to_fix is not Null AND repo_name like %s AND (started_at BETWEEN %s AND %s);'

    df = pd.read_sql_query(query,con=connection,params=[project_name,init,finish])
    #connection.close()
    #Check if column contains all values 0. In this case we do not filter outliers.
    #if checkValues(df['time_to_fix']):
    #    return df
    #else:
    #    return removeOutliers(df)
    return df

In [27]:
def computeBuildHealth(builds):
    results = list(map(lambda x: x[0], builds))
    success = list(filter(lambda x: x == True, results))
    
    if len(results) > 0:
        return len(success)/len(results)
    
    return 0

In [28]:
def updateProjectMetrics(repo_name,build_health, timeToFix, builds_actvity,coverage):
    try:
        query = """UPDATE  projects 
            set build_health = %s,
                time_to_fix_broken_builds = %s,
                builds_activity = %s,
                coverage = %s
            WHERE repo_name like %s"""

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query, [build_health, timeToFix, builds_actvity,coverage,repo_name])
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. updateProjectMetrics >>> Exception: {}".format(e)) 
        print('Project: {}'.format(repo_name))
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing updateProjectMetrics >>> Exception: {}".format(e)) 
        print('Project: {}'.format(repo_name))

In [29]:
def updateBuildTTF(project, build_number, timeToFix):
    try:
        query = """UPDATE  builds_mined 
            set time_to_fix = %s
            WHERE repo_name like %s AND build_number = %s"""

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query, [timeToFix, project, build_number])
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. updateBuildTTF >>> Exception: {}".format(e)) 
        print('Project: {}    PR - {} '.format(repo_name, pr_number))
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing updateBuildTTF >>> Exception: {}".format(e)) 
        print('Project: {}    PR - {}  '.format(repo_name, pr_number))

In [30]:
#https://www.kite.com/python/answers/how-to-remove-outliers-from-a-pandas-dataframe-in-python
def removeOutliers(df):
    z_scores = stats.zscore(df)
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    df_filtered = df[filtered_entries]

    return df_filtered

In [31]:
def checkValues(df):
    if (df == 0).all():
        return True
    else:
        i=v=0
        v = df[0]
        for a in df:
            if a != v:
                return True
            
        return False

In [32]:
def countPRsProject(project, mergeConflict=None):
    connection = connectDB()
    cursor = connection.cursor()
    
    if mergeConflict is None or mergeConflict is False:
        query = """SELECT count(id) from pullrequests WHERE project_name like %s"""
    else:
        query = """SELECT count(id) from pullrequests WHERE project_name like %s and mergeconflict is True"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [33]:
def countIssuesProject(project, bug=None):
    connection = connectDB()
    cursor = connection.cursor()
    
    if bug is None or bug is False:
        query = """SELECT count(id) from issue WHERE repo_name like %s"""
    elif bug is True:
        query = """SELECT count(id) from issue WHERE repo_name like %s and isbug is True"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [34]:
def updateProjectQty(repo_name,qtd_issues, qtd_pull_requests, qtd_bugs,qtd_merge_conflics):
    try:
        query = """UPDATE  projects 
            set qtd_issues = %s,
                qtd_pull_requests = %s,
                qtd_bugs = %s,
                qtd_merge_conflics = %s
            WHERE repo_name like %s"""

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query, [qtd_issues, qtd_pull_requests, qtd_bugs,qtd_merge_conflics,repo_name])
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. updateProjectQty >>> Exception: {}".format(e)) 
        print('Project: {}'.format(repo_name))
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing updateProjectQty >>> Exception: {}".format(e)) 
        print('Project: {}'.format(repo_name))

In [35]:
def getFirstCov(repo_name):    
    query = """SELECT min(created) FROM coverage WHERE 
            repo_name = %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return rows

In [36]:
def getFirstBuild(repo_name):    
    query = """SELECT min(started_at) FROM builds_mined WHERE 
            repo_name = %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return rows

In [37]:
def setCovInit(repo_name,cov_init,cov_init_days):
    try:
        query = """UPDATE  projects
            set cov_init = %s,
                cov_init_days = %s
            WHERE repo_name like %s;"""
            

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query,[cov_init,cov_init_days,repo_name])
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. setCovInit >>> Exception: {}".format(e)) 
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing setCovInit >>> Exception: {}".format(e)) 

In [38]:
def getNonCIProjects():
    query = """select repo_name, created from projects
            where qtd_stars >= 100 and forked is false and size  > 10000
            and ci_service is null 
            order by repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [39]:
def setProjectAnalysis_Point(repo_name,date):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """UPDATE  projects 
    set analysis_point = %s
    WHERE repo_name like %s;"""

    cursor.execute(query, [date,repo_name])
        
    connection.commit()
    cursor.close()
    connection.close()

In [40]:
def getProjectsMined():
    query = """SELECT repo_name, analysis_point From PROJECTS 
    where  qtd_stars >= 100 and forked is false and size  > 10000
            and ((ci_service is null or ci_service like 'Travis CI') and analysis_point is not null)
            order by repo_name desc;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [41]:
def setAnalysis_init_finish(repo_name,date_init,date_finish):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """UPDATE  projects 
    set analysis_init = %s,
    analysis_finish = %s
    WHERE repo_name like %s;"""

    cursor.execute(query, [date_init,date_finish,repo_name])
        
    connection.commit()
    cursor.close()
    connection.close()

In [42]:
def getMedianCIAdoption():
    query = """select  PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY ci_service_init_days) from projects
            where analysis_issues > 0 and analysis_prs > 0 and commits_mined is true AND analysis_releases > 0
                AND qty_bugs > 0 
                AND analysis_point_days is not null
                AND builds_activity > 0
                AND coverage > 0
                and ci_service ilike 'Travis CI';"""
    #AND builds_activity > 0
    #AND coverage > 0
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [43]:
def setNonCIProjectAnalysis_Point_Days(days):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """UPDATE  projects 
    set analysis_point_days = %s
    WHERE qtd_stars >= 100 
        and forked is false 
        and size  > 10000
        and ci_service is null;"""

    cursor.execute(query, [days])
        
    connection.commit()
    cursor.close()
    connection.close()

In [78]:
def getCIProjectsMetrics(ci=None):
    
    if ci is None:
        query = """select qtd_stars, qtd_issues, qtd_issues_in_period,qtd_pull_requests,qtd_pull_request_in_period,qtd_bugs,qtd_merge_conflicts,COVERAGE,BUILDS_ACTIVITY, TIME_TO_FIX_BROKEN_BUILDS, BUILD_HEALTH
                from projects where rq1 is true;"""
    elif ci is True:
        query = """select qtd_stars, qtd_issues, qtd_issues_in_period,qtd_pull_requests,qtd_pull_request_in_period,qtd_bugs,qtd_merge_conflicts,COVERAGE,BUILDS_ACTIVITY, TIME_TO_FIX_BROKEN_BUILDS, BUILD_HEALTH
                from projects where rq1 is true AND ci is True;"""
    else:
        query = """select qtd_stars, qtd_issues, qtd_issues_in_period,qtd_pull_requests,qtd_pull_request_in_period,qtd_bugs,qtd_merge_conflicts,COVERAGE,BUILDS_ACTIVITY, TIME_TO_FIX_BROKEN_BUILDS, BUILD_HEALTH
                from projects where rq1 is true AND ci is false;"""
    
    connection = connectDB()
    df = pd.read_sql_query(query,con=connection)
    connection.close()
    
    return df

In [79]:
def getBuildDates(repo_name):    
    query = """SELECT min(STARTED_at),max(STARTED_at) FROM builds_mined WHERE 
            repo_name = %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [80]:
def getCovsDates(repo_name):    
    query = """SELECT min(created),max(created) FROM coverage WHERE 
            repo_name = %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [81]:
def getBugsDates(repo_name):    
    query = """SELECT min(created_at),max(created_at) FROM issue 
            WHERE repo_name = %s AND isbug is true;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [82]:
def getReleasesDates(repo_name):    
    query = """SELECT min(created_at),max(created_at) FROM RELEASES 
            WHERE repo_name = %s AND isbug is true;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [83]:
'''projects = getAllProjects()

i=1
for proj in projects:
    project = proj[0]
    
    print('Updating project {}/{}  --  {}'.format(i,len(projects),project))
    total_prs = countPRsProject(project)
    total_merge_conflicts = countPRsProject(project,True)
    total_issues = countIssuesProject(project)
    total_bugs = countIssuesProject(project,True)
    
    updateProjectQty(project,total_issues, total_prs, total_bugs,total_merge_conflicts)
    i+=1
    
    dates_covs = getCovsDates(project)
    dates_bugs = getBugsDates(project)
    dates_releases = getReleasesDates(project)
'''

"projects = getAllProjects()\n\ni=1\nfor proj in projects:\n    project = proj[0]\n    \n    print('Updating project {}/{}  --  {}'.format(i,len(projects),project))\n    total_prs = countPRsProject(project)\n    total_merge_conflicts = countPRsProject(project,True)\n    total_issues = countIssuesProject(project)\n    total_bugs = countIssuesProject(project,True)\n    \n    updateProjectQty(project,total_issues, total_prs, total_bugs,total_merge_conflicts)\n    i+=1\n    \n    dates_covs = getCovsDates(project)\n    dates_bugs = getBugsDates(project)\n    dates_releases = getReleasesDates(project)\n"

In [29]:
#projects = getCIProjectsMetrics(ci=None)
#projects.sum()